In [1]:
import scrapy
from scrapy.crawler import CrawlerProcess
import json
import os
import string
import pandas as pd
from datetime import datetime, date, time, timedelta
import numpy as np

Inicializamos variables

In [2]:
path = '../data/urls.csv'
today = str(date.today())
days_number = 10 # Número de días que vamos a manteter las urls guardadas
scraped_urls = []

Leemos el fichero con las urls guardadas y eliminamos las que tengan mas días de los estipulados

In [3]:
df_urls = pd.read_csv(path)
df_urls['date'] = pd.to_datetime(df_urls['date'], format="%Y/%m/%d")
dates = df_urls['date']
keep = dates > (np.datetime64(date.today() - timedelta(days=days_number)))
df_urls = df_urls[keep]
scraped_urls = list(df_urls['url'])
df_urls.head()

,url,date
0,https://www.20minutos.es/noticia/4369249/0/ayu...,2020-09-04
1,https://www.20minutos.es/noticia/4367265/0/map...,2020-09-04
2,https://www.20minutos.es/noticia/4369032/0/asm...,2020-09-04
3,https://www.20minutos.es/noticia/4369272/0/can...,2020-09-04
4,https://www.20minutos.es/noticia/4369279/0/con...,2020-09-04


Creamos la clase Spider que recorrerá las páginas que pasamos en la variable start_urls

In [4]:
class url_spider(scrapy.Spider):
    name = 'blogspider'
    start_urls = [
        'https://www.20minutos.es/',
        'https://www.larazon.es/',
        'https://www.publico.es/',
        'https://www.abc.es/',
        'https://www.lesoir.be/',
        'https://www.news.com.au/',
        'https://time.com/',
        'https://elpais.com/',
        'https://www.elmundo.es/'
    ]

    
    def parse(self, response):

        global df_urls
        global scraped_urls

        urls = []

        # Comprobamos de donde viene la url para hacer el scraping según el periódico del que se trate.
        # Comprobamos que sa url no este anteriormente scrapeada

        if response.url.startswith('https://www.20minutos.es'):
            for article in response.css('article'):
                url = article.css('header h1 a::attr(href)').extract_first()
                if url.startswith('https://www.20minutos.es') and url not in scraped_urls:
                    urls.append(url)
                
        elif response.url.startswith('https://www.larazon.es/'):
            urls_la_razon = set(response.css('a.story-card__areas__head::attr(href)').getall())
            for url in urls_la_razon:
                if url not in scraped_urls:
                    urls.append(url)

        elif response.url.startswith('https://www.publico.es/'):
            urls_publico = set(response.css('a.page-link::attr(href)').getall())
            for url in urls_publico:
                if not url.startswith('http'):
                    url = 'https://www.publico.es' + str(url)
                    if url not in scraped_urls:
                        urls.append(url)

        elif response.url.startswith('https://www.elmundo.es'):
            urls_mundo = set(response.css('a.ue-c-cover-content__link::attr(href)').getall())
            for url in urls_mundo:
                if url is not None:
                    if not url.startswith('http'):
                        url = 'https://www.elmundo.es' + str(url)
                    if url.startswith('https://www.elmundo.es') and url not in scraped_urls:
                        urls.append(url)
        
        elif response.url.startswith('https://elpais.com'):
            for article in response.css('h2.headline'):
                url = article.css('a::attr(href)').extract_first()
                if url is not None:
                    if not url.startswith('http'):
                        url = 'https://elpais.com' + str(url)
                        if url not in scraped_urls:
                            urls.append(url)

        elif response.url.startswith('https://time.com'):
            for article in response.css('h2.title'):
                url = str(article.css('a::attr(href)').extract_first())
                if not url.startswith('https:'):
                    url = 'https://time.com/' + url
                if url.startswith('https://time.com') and url not in scraped_urls:
                    urls.append(url)
                        
        elif response.url.startswith('https://www.abc.es/'):
            for article in response.css('article'):
                url = article.css('span h3 a::attr(href)').extract_first()
                if url is not None:
                    if not url.startswith('http'):
                        url = 'https://www.abc.es' + str(url)
                    if url.startswith('https://www.abc.es') and url not in scraped_urls:
                        urls.append(url)
                  
        elif response.url.startswith('https://www.lesoir.be/'):
            for article in response.css('h4.media-heading'):
                url = article.css('a::attr(href)').extract_first()
                if url is not None:
                    if not url.startswith('http'):
                        url = 'https://www.lesoir.be' + str(url)
                    if url.startswith('https://www.lesoir.be') and url not in scraped_urls:
                            urls.append(url)
                  
        elif response.url.startswith('https://www.news.com.au/'):
            for article in response.css('h4.heading'):
                url = article.css('a::attr(href)').extract_first()
                if url is not None:
                    if not url.startswith('http'):
                        url = 'https://www.news.com.au/' + str(url)
                    if url.startswith('https://www.news.com') and url not in scraped_urls:
                            urls.append(url)
            
        # Agregamos las urls al dataframe

        for url in urls:
            df_urls = df_urls.append({'url' : url , 'date' : today} , ignore_index=True)

        # Mostramos las últimas urls scrpaeadas
        df_urls.tail()

Lanzamos el proceso de Crawler

In [5]:
process = CrawlerProcess({
    'USER_AGENT': 'Google SEO Bot'
})
process.crawl(url_spider)
process.start()

2020-09-10 18:14:17 [scrapy.utils.log] INFO: Scrapy 2.3.0 started (bot: scrapybot)
2020-09-10 18:14:17 [scrapy.utils.log] INFO: Versions: lxml 4.5.0.0, libxml2 2.9.10, cssselect 1.1.0, parsel 1.6.0, w3lib 1.22.0, Twisted 20.3.0, Python 3.7.6 (default, Jan  8 2020, 13:42:34) - [Clang 4.0.1 (tags/RELEASE_401/final)], pyOpenSSL 19.1.0 (OpenSSL 1.1.1g  21 Apr 2020), cryptography 2.9.2, Platform Darwin-19.6.0-x86_64-i386-64bit
2020-09-10 18:14:17 [scrapy.utils.log] DEBUG: Using reactor: twisted.internet.selectreactor.SelectReactor
2020-09-10 18:14:17 [scrapy.crawler] INFO: Overridden settings:
{'USER_AGENT': 'Google SEO Bot'}
2020-09-10 18:14:17 [scrapy.extensions.telnet] INFO: Telnet Password: e5f574c21111d0f1
2020-09-10 18:14:17 [scrapy.middleware] INFO: Enabled extensions:
['scrapy.extensions.corestats.CoreStats',
 'scrapy.extensions.telnet.TelnetConsole',
 'scrapy.extensions.memusage.MemoryUsage',
 'scrapy.extensions.logstats.LogStats']
2020-09-10 18:14:17 [scrapy.middleware] INFO: Enab

In [6]:
# Mostramos las últimas urls scrapeadas
df_urls.tail()

,url,date
3489,https://www.news.com.au/finance/work/at-work/r...,2020-09-10
3490,https://www.news.com.au/finance/business/retai...,2020-09-10
3491,https://www.news.com.au/sport/afl/west-coast-p...,2020-09-10
3492,https://www.news.com.au/sport/american-sports/...,2020-09-10
3493,https://www.news.com.au/sport/afl/footy-fans-l...,2020-09-10
